In [1]:
import pandas as pd

import geopandas as gpd
from shapely.geometry import Point
from geopy.distance import distance

In [2]:
# Función para calcular la distancia entre dos puntos
def calcular_distancia(lat1, lon1, lat2, lon2):
    return distance((lat1, lon1), (lat2, lon2)).km

In [3]:
df = pd.read_csv('data/co.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
df['ci_es'] = df['city'] + ' - ' + df['admin_name']
df = df.drop_duplicates(subset=['ci_es'])
df.head()

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper,ci_es
0,Bogotá,4.6126,-74.0705,Colombia,CO,Bogotá,primary,9464000.0,7963000.0,Bogotá - Bogotá
1,Medellín,6.2447,-75.5748,Colombia,CO,Antioquia,admin,2529403.0,2529403.0,Medellín - Antioquia
2,Cali,3.4400,-76.5197,Colombia,CO,Valle del Cauca,admin,2471474.0,2471474.0,Cali - Valle del Cauca
3,Barranquilla,10.9639,-74.7964,Colombia,CO,Atlántico,admin,1274250.0,1274250.0,Barranquilla - Atlántico
4,Cartagena,10.4236,-75.5253,Colombia,CO,Bolívar,admin,1036412.0,1036412.0,Cartagena - Bolívar


In [4]:
base = df.copy()
base = base.loc[:,['ci_es','lat','lng']]
base.head()

,ci_es,lat,lng
0,Bogotá - Bogotá,4.6126,-74.0705
1,Medellín - Antioquia,6.2447,-75.5748
2,Cali - Valle del Cauca,3.4400,-76.5197
3,Barranquilla - Atlántico,10.9639,-74.7964
4,Cartagena - Bolívar,10.4236,-75.5253


In [5]:
origen = base.copy()
origen = origen.loc[:,['ci_es']]
origen.insert(1, 'count', 1)
origen['city_list'] = origen['count'].apply(lambda x: list(origen['ci_es']))
origen = origen.explode('city_list')
origen.rename(columns={'ci_es':'from', 'city_list':'to'}, inplace=True)
origen = origen.loc[:,['from','to']]
origen.head()

,from,to
0,Bogotá - Bogotá,Bogotá - Bogotá
0,Bogotá - Bogotá,Medellín - Antioquia
0,Bogotá - Bogotá,Cali - Valle del Cauca
0,Bogotá - Bogotá,Barranquilla - Atlántico
0,Bogotá - Bogotá,Cartagena - Bolívar


In [6]:
origen.shape

(1210000, 2)

In [7]:
distances = pd.merge(origen, base, how='left', left_on='from', right_on='ci_es')  
distances.rename(columns={'lat':'latitud_from', 'lng':'longitud_from'}, inplace=True)
distances = pd.merge(distances, base, how='left', left_on='to', right_on='ci_es') 
distances.rename(columns={'lat':'latitud_to', 'lng':'longitud_to'}, inplace=True)
distances = distances.loc[:,['from', 'to', 'latitud_from', 'longitud_from', 'latitud_to', 'longitud_to']]
distances.shape

(1210000, 6)

In [8]:
distances['distancia'] = distances.apply(lambda row: calcular_distancia(row['latitud_from'], row['longitud_from'], row['latitud_to'], row['longitud_to']), axis=1)

In [9]:
distances.head()

,from,to,latitud_from,longitud_from,latitud_to,longitud_to,distancia
0,Bogotá - Bogotá,Bogotá - Bogotá,4.6126,-74.0705,4.6126,-74.0705,0.000000
1,Bogotá - Bogotá,Medellín - Antioquia,4.6126,-74.0705,6.2447,-75.5748,245.694109
2,Bogotá - Bogotá,Cali - Valle del Cauca,4.6126,-74.0705,3.4400,-76.5197,301.299069
3,Bogotá - Bogotá,Barranquilla - Atlántico,4.6126,-74.0705,10.9639,-74.7964,706.970469
4,Bogotá - Bogotá,Cartagena - Bolívar,4.6126,-74.0705,10.4236,-75.5253,662.399801


In [13]:
distances.to_csv('data/distances.csv', encoding = 'utf-8-sig', index = False)
distances.to_parquet('data/distances.parquet.gzip', compression='gzip')

In [11]:
# import pandas as pd

# data = {'a': [1, 2, 3, 4, 5], 'b': [10, 20, 30, 40, 50]}
# df = pd.DataFrame(data)

# df['a_list'] = df['b'].apply(lambda x: list(df['a']))
# # df = df.explode('a_list')

# print(df)

In [12]:
print('ok')

ok
